In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as soup
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime

In [2]:
#開啟google
browser = webdriver.Chrome('./chromedriver')
#設定要前往的網址
url = 'https://www.google.com.tw/maps?hl=zh-TW&tab=rl&authuser=0'  
#前往該網站
browser.get(url)

In [3]:
#設立搜尋字元
search = '法朋'

In [4]:
#填入網站搜尋內容
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'tactile-searchbox-input')))
#網頁元素定位
search_input = browser.find_elements_by_class_name('tactile-searchbox-input')[0]
#輸入搜尋內容
search_input.send_keys(search)
#搜尋
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'searchbox-searchbutton-container')))
#網頁元素定位
search_click = browser.find_elements_by_class_name('searchbox-searchbutton-container')[0]
#點下去~
search_click.click()


In [75]:
# 使用方法:
# 1.找到有問題的店家所在那頁
# 2.建立此清單
# 3.並依據此清單檢視未爬取完成之項目

# ex.我在P.2的第3家(name_ls[2])出現有問題的店家，則用原本輕鬆版本的function將無法操作且後面第4到第20家將無法爬取，故到這邊以分開的方法先建立清單，並根據清單裡未爬取的店家逐一爬取

# 網頁元素定位
web_out=soup(browser.page_source,"lxml")
# 店家名字清單
name=web_out.find_all("h3",class_="section-result-title")
name_ls=[]
for i in name:
    name_ls.append(i.text)

name_ls

[]

In [65]:
search_click = browser.find_elements_by_class_name('section-result-title')[]#  這邊需填寫你想爬的店家在清單裡面屬於第幾個
search_click.click()
WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-result-title')))
web_in =soup(browser.page_source,"lxml")
address_ls=[]
address = web_in.find_all('button', {'data-tooltip':"複製地址"})
address_ls.append(address[0].get('aria-label').strip().strip('地址: '))
# 下拉評論
WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "gm2-button-alt.jqnFjrOWMVU__button-blue")))
pane= browser.find_elements_by_class_name("gm2-button-alt.jqnFjrOWMVU__button-blue")[0]
action=ActionChains(browser).move_to_element(pane).perform()
pane.click()
time.sleep(2)
# 打開評論選單
WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[7]/div[2]/div')))
menu_click = browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[7]/div[2]/div')
menu_click.click()
time.sleep(2)
# 選擇最新
category_click = browser.find_elements_by_class_name('action-menu-entry')[1]
category_click.click()

In [5]:
# 若不想操作前面兩段也可以手動操作點完上述操作並執行這段(若前面有執行則直接執行即可)，直接手動操作比較針對像是法朋這種搜尋結果只有一家店而非一個店家清單
today = datetime.today().strftime("%Y-%m-%d")
st= 0 # 看你要滑幾次
stop = False
rev_num=[0]
while True:
    date_ls = []
    pane = browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
    date = browser.find_elements_by_class_name('section-review-publish-date')
    time.sleep(1)
    for item in date:
        date_ls.append(item.text)
    for it in date_ls:
        if it == '2 年前':
            stop = True
            break# 若遇到2年前字樣則跳出
    st = st + 1
    if stop == True:
        break
    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
    rev = soup(browser.page_source,"lxml")
    rev_len=rev.find_all(class_ = 'section-review ripple-container GLOBAL__gm2-body-2')
    rev_num.append(len(rev_len))
    if rev_num[-1] == rev_num[-2]:
        break# 針對有些新創店家評論不到2年前，在此根據評論長度判斷是否跳出
    #
    print(">>正在爬取評論...下滑次數："+str(st))
    time.sleep(1)
# 爬取所有評論(此時會包含到2年前，這邊為爬取建到2年前字樣停止滾動當下所有的評論)
re = soup(browser.page_source,"lxml")
all_reviews = re.find_all(class_ = 'section-review ripple-container GLOBAL__gm2-body-2')
#
all_star_review = []
all_date_review = []
all_text_review = []

for ar in all_reviews:
     # 評論星數
    all_star_review.append(str(ar.find(class_ = "section-review-stars").get('aria-label').strip().strip("顆星")))
    # 評論時間
    all_date_review.append(ar.find(class_ = "section-review-publish-date").text)
    # 評論內容
    all_text_review.append(ar.find(class_ = "section-review-text").text)
#
reviews = pd.DataFrame({
    'Store':"Le Ruban Pâtisserie 法朋烘焙甜點坊",# 需設定順序，若未執行上面兩段則需手動命名
    'Today':today,
    'Star Reviews':all_star_review,
    'Date':all_date_review,
    'Text':all_text_review,
    'Address':address_ls[0] # 若未執行上面兩段則需手動命名
    })
reviews = reviews[reviews['Date'] != '2 年前']
reviews = reviews[reviews['Date'] != '3 年前']
#

>>正在爬取評論...下滑次數：1
>>正在爬取評論...下滑次數：2
>>正在爬取評論...下滑次數：3
>>正在爬取評論...下滑次數：4
>>正在爬取評論...下滑次數：5
>>正在爬取評論...下滑次數：6
>>正在爬取評論...下滑次數：7
>>正在爬取評論...下滑次數：8
>>正在爬取評論...下滑次數：9
>>正在爬取評論...下滑次數：10
>>正在爬取評論...下滑次數：11
>>正在爬取評論...下滑次數：12
>>正在爬取評論...下滑次數：13
>>正在爬取評論...下滑次數：14
>>正在爬取評論...下滑次數：15
>>正在爬取評論...下滑次數：16
>>正在爬取評論...下滑次數：17
>>正在爬取評論...下滑次數：18
>>正在爬取評論...下滑次數：19
>>正在爬取評論...下滑次數：20
>>正在爬取評論...下滑次數：21
>>正在爬取評論...下滑次數：22
>>正在爬取評論...下滑次數：23
>>正在爬取評論...下滑次數：24
>>正在爬取評論...下滑次數：25
>>正在爬取評論...下滑次數：26
>>正在爬取評論...下滑次數：27
>>正在爬取評論...下滑次數：28
>>正在爬取評論...下滑次數：29
>>正在爬取評論...下滑次數：30
>>正在爬取評論...下滑次數：31
>>正在爬取評論...下滑次數：32
>>正在爬取評論...下滑次數：33
>>正在爬取評論...下滑次數：34
>>正在爬取評論...下滑次數：35
>>正在爬取評論...下滑次數：36
>>正在爬取評論...下滑次數：37
>>正在爬取評論...下滑次數：38
>>正在爬取評論...下滑次數：39
>>正在爬取評論...下滑次數：40
>>正在爬取評論...下滑次數：41
>>正在爬取評論...下滑次數：42
>>正在爬取評論...下滑次數：43
>>正在爬取評論...下滑次數：44
>>正在爬取評論...下滑次數：45
>>正在爬取評論...下滑次數：46
>>正在爬取評論...下滑次數：47
>>正在爬取評論...下滑次數：48
>>正在爬取評論...下滑次數：49
>>正在爬取評論...下滑次數：50
>>正在爬取評論...下滑次數：51
>>正在爬取評論...下滑次數：52
>>正在爬取評論...下滑次數：53
>>

In [7]:
# 輸出csv
# 填寫此次爬取順序或手動設定檔名(未執行前述提及兩段)
reviews.to_csv("Le Ruban Pâtisserie 法朋烘焙甜點坊_所有評論.csv", encoding = 'UTF-8-sig',index=False)

In [ ]:
# 返回原本頁面 # 但基本上都到這邊了，其實也可以手動
BacktoStore_btn = browser.find_elements_by_class_name('mdc-button__icon')[0]
BacktoStore_btn.click()
WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-back-to-list-button.blue-link.noprint')))
back_btn = browser.find_elements_by_class_name('section-back-to-list-button.blue-link.noprint')[0]
back_btn.click()

In [ ]:
# 如果運氣超好 剛好後面的資料都沒有出現有問題的或一些不該出現的結果(ex.亞尼克的捷運提領站...)
# 那你就可以搭配前面的name_ls(不用執行它底下那段)，針對後面特定的連續數目(ex.3~20或是短短的3~7)，去做爬取

for n in range(3:20): # 根據當頁缺乏的店家，有點忘記range那邊是不是這樣設定了
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-result-title'))) # 等待該格load進去後才執行後續
    search_click = browser.find_elements_by_class_name('section-result-title')[n] # 從原本頁面點進該店家
    search_click.click()
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-result-title')))
    web_in =soup(browser.page_source,"lxml")
    address_ls=[]
    address = web_in.find_all('button', {'data-tooltip':"複製地址"})
    address_ls.append(address[0].get('aria-label').strip().strip('地址: '))
    #下拉評論並點開所有評論
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "gm2-button-alt.jqnFjrOWMVU__button-blue")))
    pane= browser.find_elements_by_class_name("gm2-button-alt.jqnFjrOWMVU__button-blue")[0]
    action=ActionChains(browser).move_to_element(pane).perform()
    pane.click()
    time.sleep(2)
    # 點開評論選單
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[7]/div[2]/div')))
    menu_click = browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[7]/div[2]/div')
    menu_click.click()
    time.sleep(1)
    # 選擇評論類型，在此選擇最新選項
    category_click = browser.find_elements_by_class_name('action-menu-entry')[1]
    category_click.click()
    st= 0 # 看你要滑幾次
    stop = False
    rev_num=[0]
    while True:
        # 建立滾輪下拉loop
        date_ls = []
        pane = browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
        date = browser.find_elements_by_class_name('section-review-publish-date')
        time.sleep(1)
        for item in date:
            date_ls.append(item.text)
        for it in date_ls:
            if it == '2 年前':
                stop = True
                break# 若遇到2年前字樣則跳出
        st = st + 1
        if stop == True:
            break
        browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
        rev = soup(browser.page_source,"lxml")
        rev_len=rev.find_all(class_ = 'section-review ripple-container GLOBAL__gm2-body-2')
        rev_num.append(len(rev_len))
        print(len(rev_num))
        if rev_num[-1] == rev_num[-2]:
            break# 針對有些新創店家評論不到2年前，在此根據評論長度判斷是否跳出
    print(">>正在爬取評論...下滑次數："+str(st))
    time.sleep(1)
    # 爬取所有評論(此時會包含到2年前，這邊為爬取建到2年前字樣停止滾動當下所有的評論)
    re = soup(browser.page_source,"lxml")
    all_reviews = re.find_all(class_ = 'section-review ripple-container GLOBAL__gm2-body-2')
    #抓取評論
    all_star_review = []
    all_date_review = []
    all_text_review = []
    for ar in all_reviews:
        # 評論星數
        all_star_review.append(str(ar.find(class_ = "section-review-stars").get('aria-label').strip().strip("顆星")))
        # 評論時間
        all_date_review.append(ar.find(class_ = "section-review-publish-date").text)
        # 評論內容
        all_text_review.append(ar.find(class_ = "section-review-text").text)
    #建立dataframe
    reviews = pd.DataFrame({
        'Store':name_ls[n], #記得name_ls一定要設定好喔
        'Today':today,
        'Star Reviews':all_star_review,
        'Date':all_date_review,
        'Text':all_text_review,
        'Address':address_ls[0]
        })
    reviews = reviews[reviews['Date'] != '2 年前'] #在此刪去
    reviews = reviews[reviews['Date'] != '3 年前'] #有時會1年前跳3年前故保險起見額外增加此行

    # 輸出(這邊就先丟外面再自己分類一下比較不會搞混)
    reviews.to_csv(name_ls[n] +"_所有評論.csv", encoding = 'UTF-8-sig',index=False)
    # 回個別介面
    time.sleep(2)
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'mdc-button__icon')))
    BacktoStore_btn = browser.find_elements_by_class_name('mdc-button__icon')[0]
    BacktoStore_btn.click()
    # 返回搜尋結果
    time.sleep(2)
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-back-to-list-button.blue-link.noprint')))
    back_btn = browser.find_elements_by_class_name('section-back-to-list-button.blue-link.noprint')[0]
    back_btn.click()
    time.sleep(2)
    # 爬完後後面幾頁若沒問題則可以跳回輕鬆function做爬取